### 사진 화이트 밸런스 변환기
    
    최초 작성일 : 20/02/09
    작성자 : 양희승
    
    작성내용 : 사진의 빛이 다르기 때문에 하나로 정리할 화이트 밸런스 변환하는 코드
    
    
    수정내용
        20/02/18
            테스트 사진 추가
            
            
        20/02/22
            색상 보정

In [1]:
# 설치
#  sudo apt-get install python-opencv  
# !pip install opencv-python  #메인모듈만
# !pip install opencv-contrib-python  #기본 및 contrib 모듈이 모두 필요한 경우

# 임포트가 안된다면
# sudo -H python -m pip install opencv-python 
import cv2 as cv
import numpy as np

In [6]:
def show(final):
    print('display')
    cv.imshow("Temple", final)
    cv.waitKey(0)
    cv.destroyAllWindows()

def saveimg(final) :
    print('saving')
    cv.imwrite("result.jpg", final)

In [7]:
# #Insert anyfilenmae with path
# img = cv.imread("img/1.jpg")
# res = img
# final = cv.cvtColor(res, cv.COLOR_BGR2LAB)


# avg_a = -np.average(final[:,:,1])
# avg_b = -np.average(final[:,:,2])

# for x in range(final.shape[0]) :
#     for y in range(final.shape[1]) :
#         l, a, b = final[x][y]
#         shift_a =avg_a * (1/100.0) * 1.1
#         shift_b =avg_b * (1/100.0) * 1.1
#         final[x][y][1]= a + shift_a
#         final[x][y][2]= b + shift_b
        
# final = cv.cvtColor(final, cv.COLOR_LAB2BGR)
# final = np.hstack((res, final))
# show(final)
# saveimg(final)


In [6]:
def wb_conversion(img) :
    result = cv.cvtColor(img, cv.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv.cvtColor(result, cv.COLOR_LAB2BGR)
    return result

In [11]:
#Insert anyfilenmae with path
img = cv.imread("img/4226019_130.jpg")
wb_img = wb_conversion(img)
h, w, channel = wb_img.shape
print("높이 : ",h)
print("가로 : ",w)
print("채널 : ",channel)
# if h > w :
#     final = cv.resize(wb_img, dsize=(540, 720), interpolation=cv.INTER_AREA)
# elif h < w :
#     final = cv.resize(wb_img, dsize=(720, 540), interpolation=cv.INTER_AREA)
# else :
#     final = cv.resize(wb_img, dsize=(540, 540), interpolation=cv.INTER_AREA)

cv.imwrite('img/4226019_130_convert.jpg', wb_img)

높이 :  349
가로 :  620
채널 :  3


True

## 테스트 사진 코드

In [7]:
file_name = "01"
img = cv.imread("img/"+file_name+".jpg")
wb_img = wb_conversion(img)
cv.imwrite("img/"+file_name+"_wb.jpg", wb_img)

True

## 새 테스트

In [2]:
import math
import numpy as np
import sys

def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100

    half_percent = percent / 200.0

    channels = cv.split(img)

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)

        assert len(flat.shape) == 1

        flat = np.sort(flat)

        n_cols = flat.shape[0]

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        print("Lowval: ", low_val)
        print("Highval: ", high_val)

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv.normalize(thresholded, thresholded.copy(), 0, 255, cv.NORM_MINMAX)
        out_channels.append(normalized)

    return cv.merge(out_channels)

In [3]:
file_name = "origin"
img = cv.imread("img/"+file_name+".jpg")
out = simplest_cb(img, 1)
cv.imwrite("img/"+file_name+"_wb1.jpg", out)
cv.imshow("before", img)
cv.imshow("after", out)

# ex = cv.imread("img/"+file_name+"_wb2.jpg")
# cv.imshow("ex", ex)
cv.waitKey(0)

Lowval:  38
Highval:  242
Lowval:  22
Highval:  238
Lowval:  16
Highval:  239


-1